### Script adapté de Waveimage-tensorflow-pow3-encoding_P3ver.ipynb afin d'y appliquer les méthodes de la librairie TensorFlow

In [1]:
%matplotlib inline

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import math

In [3]:
import pywt # wavelets transforms library

In [4]:
# Telecharger des exemples et leurs labels provenant de la base MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
# Fonction permettant de modifier la taille et l'emplacement du stimulus dans l'image
def mnist_reshape_128(x, i_offset = 0, j_offset = 0): # i,j : coordonnees du stimulus par rapport au centre de l'image
    assert x.shape == (28 * 28,)
    image = x.reshape(28, 28)
    image = np.append(np.zeros((128 + 2, 28)), image, axis = 0)
    image = np.append(image, np.zeros((128 + 2, 28)), axis = 0)
    image = np.append(np.zeros((288, 128 + 2)), image, axis = 1)
    image = np.append(image, np.zeros((288, 128 + 2)), axis = 1)
    return image[128 + 16 - 64 - i_offset : 128 + 16 + 64 - i_offset, 128 + 16 - 64 - j_offset : 128 + 16 + 64 - j_offset]

In [6]:
 # Fonction calculant le nombre d'ondelettes en fonction de l'echelle ? ; shape : taille image
def calc_dim(shape, h, h_max):
    assert 0 <= h < h_max # h : echelle de composition d'ondelette
    if h == 0:
        dim_i = int(math.ceil(shape[0] * 1. // 2**(h_max - 1)))
        dim_j = int(math.ceil(shape[1] * 1. // 2**(h_max - 1)))
    else :
        dim_i = int(math.ceil(shape[0] * 1. // 2**(h_max - h)))
        dim_j = int(math.ceil(shape[1] * 1. // 2**(h_max - h)))
    return dim_i, dim_j

In [7]:
class WaveImage:
	
	def __init__(self, image = None, shape = (32, 32)):
		
		# Attribut shape
		if image is not None:
			# Decomposition ondelettes
			coeffs = pywt.wavedec2(image, 'haar') # haar correspond à une famille d'ondelettes
			self.__shape = image.shape
		else:
			self.__shape = shape
		
		# Attribut h_max : profondeur de l'image
		self.__h_max = min(int(math.log(self.__shape[0], 2)) + 1, 	int(math.log(self.__shape[1], 2)) + 1)
			
		# Attribut data : L'attribut data contient les vecteurs en position [h][u] (dictionnaire)
		if image is not None:
			self.__data = {}
			for h in range(self.__h_max):
				self.__data[h] = {}
				if h == 0:
					(i_max, j_max) = coeffs[h].shape
				else:
					(i_max, j_max) = coeffs[h][0].shape
				for i in range(i_max):
					for j in range(j_max):
						if h == 0:
							data = coeffs[h][i][j]
						else:
							data = coeffs[h][0][i][j] # k : num ondelette ?
							for k in range(1,len(coeffs[h])):
								data = np.append(data, coeffs[h][k][i][j])	
						self.__data[h][(i, j)] = data				
		else: # image is None
			self.__data = {}
			for h in range(self.__h_max):
				self.__data[h] = {}
					
		
	def get_data(self):
		return self.__data
	
	def get_shape(self):
		return self.__data
				
	def set_data(self, h, u, v):
		assert 0 <= h < self.__h_max
		dim_i, dim_j = calc_dim(self.__shape, h, self.__h_max)
		assert 0 <= u[0] < dim_i
		assert 0 <= u[1] < dim_j
		if h == 0 :
			self.__data[h][u] = v
		else:
			self.__data[h][u] = np.copy(v)
		
	def get_h_max(self):
		return self.__h_max
		
	def get_image(self): # etape decodage pour retrouver image
		coeffs = []
		for h in range(self.__h_max):
			dim_i, dim_j = calc_dim(self.__shape, h, self.__h_max)
			if h == 0:
				coeffs_h = np.zeros((dim_i, dim_j))
				for u in self.__data[h]:
					coeffs_h[u[0],u[1]] = self.__data[h][u]
			else:
				coeffs_h = [np.zeros((dim_i, dim_j)), np.zeros((dim_i, dim_j)), np.zeros((dim_i, dim_j))]
				for u in self.__data[h]:
					for k in range(3):
						coeffs_h[k][u[0],u[1]] = self.__data[h][u][k]
			coeffs += [coeffs_h]
		return pywt.waverec2(coeffs, 'haar')
		
	def add_coeffs(self, waveImage, u, h_ref = 0): # copie certains niveaux de l'arbre d'ondelettes
		# Niveau 0
		h_opp = self.__h_max - 1
		i = int(u[0] // 2**h_opp) 
		j = int(u[1] // 2**h_opp)
		u_0 = (i,j)
		if self.__data[0] == {}:
			self.__data[0][u_0] = waveImage.get_data()[0][u_0]
		else:
			v_test = self.__data[0][u_0]
			if np.linalg.norm(v_test) < 1e-16:
				self.__data[0][u_0] = waveImage.getData()[0][u_0]
		# Niveaux 1 et +
		for h in range(1, h_ref) :
			h_opp = self.__h_max - h
			i = int(u[0] // 2**h_opp) 
			j = int(u[1] // 2**h_opp)
			if (i,j) in self.__data[h]:
				v_test = self.__data[h][(i,j)]
				if np.linalg.norm(v_test) < 1e-16:
					self.__data[h][(i,j)] = np.copy(waveImage.get_data()[h][(i,j)])
			else: 
				self.__data[h][(i,j)] = np.copy(waveImage.get_data()[h][(i,j)])
	
	def copy(self): # copie complete
		self_shape = self.__shape 
		self_copy = WaveImage(shape = self_shape)
		for h in range(self.__h_max) :
			for u in self.__data[h]:
				self_copy.set_data(h, u, self.__data[h][u])
		return self_copy	
		
	def __str__(self): # affiche contenu objet
		h_max = len(self.__data)
		s = 'h_max :' + str(self.__h_max) + '\n'
		for h in range(self.__h_max):
			s += '***' + str(h) + '***\n'
			s += str(self.__data[h]) + '\n'
		return s

In [8]:
# Genere un dictionnaire de tenseurs indexes par la profondeur dans l'arbre des coefficients d'ondelettes 
def generate_tensor_data_with_offset_from_x(x, i_offset, j_offset):
    w1 = WaveImage(shape = (128, 128))
    w2 = WaveImage(image = mnist_reshape_128(x, i_offset = i_offset, j_offset = j_offset))
    w1.add_coeffs(w2, u = (63, 63), h_ref = w2.get_h_max())
    w1.add_coeffs(w2, u = (63, 65), h_ref = w2.get_h_max())
    w1.add_coeffs(w2, u = (65, 63), h_ref = w2.get_h_max())
    w1.add_coeffs(w2, u = (65, 65), h_ref = w2.get_h_max())
    h_max = w1.get_h_max()
    data = w1.get_data()
    tensor_data = {}
    for k in data :
        if k == 0:
            tensor_data[0] = data[k][(0, 0)]
        elif k == 1:
            tensor_data[1] = np.array(data[k][(0, 0)])    
        else:
            tensor_data[k] = np.zeros((2, 2, 3))
            for u in data[k]:           
                u_offset = 64 // (2**(h_max - k)) - 1
                tensor_data[k][u[0] - u_offset, u[1] - u_offset, :] = np.array(data[k][u])
    return tensor_data, w1

In [9]:
def generate_vector_data_with_offset_from_x(x, i_offset, j_offset):
    # retourne un vecteur contenant les coefficients utilisés de l'image w1 générée à partir d'un point de fixation
    # central avec la cible en position i_offset, j_offset
    w1 = WaveImage(shape = (128, 128))
    w2 = WaveImage(image = mnist_reshape_128(x, i_offset = i_offset, j_offset = j_offset))
    w1.add_coeffs(w2, u = (63, 63), h_ref = w2.get_h_max())
    w1.add_coeffs(w2, u = (63, 65), h_ref = w2.get_h_max())
    w1.add_coeffs(w2, u = (65, 63), h_ref = w2.get_h_max())
    w1.add_coeffs(w2, u = (65, 65), h_ref = w2.get_h_max())
    h_max = w1.get_h_max() # h = 7
    data = w1.get_data()
    vector_data = np.array([])
    for k in data :
        if k == 0:
            vector_data = np.append(vector_data, [data[k][(0, 0)]])
        elif k == 1:
            vector_data = np.append(vector_data, data[k][(0, 0)])  
        else:
            for u in data[k]:           
                 vector_data = np.append(vector_data, data[k][u])
    return vector_data, w1

In [16]:
def minmax(value,   #valeur a delimiter
           border): #limite min/max a ne pas depasser 
    value = max(value, -border)
    value = min(value, border)
    return value

### Intégration d'un régression linéaire via TF

In [12]:
import tensorflow as tf

In [13]:
sess = tf.Session()

In [106]:
batch_size = 1000  # taille de l'échantillon
iterations = 1000 # nombre d'itérations à réaliser dans l'apprentissage

In [118]:
# Définir la liste de features
feature_columns = [tf.feature_column.numeric_column("x", shape=[76])]

# Définir le type d'estimateur utilisé
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns, label_dimension=2)

### Définition des jeux de données
# Récuperation des données pour la phase d'apprentissage
batch = mnist.train.next_batch(batch_size)
x_train, y_train = [], []

for x in batch[0]:
    # i, j correspondent aux coordonnées réelles de la cible, comprises dans l'intervalle (-40,40)
    i_offset, j_offset = minmax(int(np.random.randn() * 15), 40), minmax(int(np.random.randn() * 15), 40)  
    v, w = generate_vector_data_with_offset_from_x(x, i_offset, j_offset) # v: vector, utilise comme x
    x_train += [(v)]
    y_train += [(i_offset, j_offset)]

# Récupération des données pour la phase d'évaluation
batch = mnist.test.next_batch(batch_size)
x_eval, y_eval = [], []

for x in batch[0]:
    # i, j correspondent aux coordonnees reelles de la cible, comprises dans l'intervalle (-40,40)
    i_offset, j_offset = minmax(int(np.random.randn() * 15), 40), minmax(int(np.random.randn() * 15), 40)  
    v, w = generate_vector_data_with_offset_from_x(x, i_offset, j_offset) # v: vector, utilise comme x
    x_eval += [(v)]
    y_eval += [(i_offset, j_offset)]
    
input_fn = tf.estimator.inputs.numpy_input_fn({"x" : np.array(x_train)},
                                              np.array(y_train),
                                              batch_size=batch_size,
                                              num_epochs=None,
                                              shuffle=True)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_service': None, '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fea3ec2a898>, '_is_chief': True, '_master': '', '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_task_id': 0, '_model_dir': '/tmp/tmp4stt7pgs', '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_tf_random_seed': None, '_session_config': None, '_save_summary_steps': 100, '_task_type': 'worker', '_keep_checkpoint_max': 5}


INFO:tensorflow:Using config: {'_service': None, '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fea3ec2a898>, '_is_chief': True, '_master': '', '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_task_id': 0, '_model_dir': '/tmp/tmp4stt7pgs', '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_tf_random_seed': None, '_session_config': None, '_save_summary_steps': 100, '_task_type': 'worker', '_keep_checkpoint_max': 5}


In [119]:
# Lancement de l'entrainement
estimator.train(input_fn=input_fn, steps=iterations)

INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp4stt7pgs/model.ckpt.


INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp4stt7pgs/model.ckpt.


INFO:tensorflow:loss = 445594.0, step = 1


...

INFO:tensorflow:loss = 79914.1, step = 901 (0.560 sec)


INFO:tensorflow:Saving checkpoints for 1000 into /tmp/tmp4stt7pgs/model.ckpt.


INFO:tensorflow:Saving checkpoints for 1000 into /tmp/tmp4stt7pgs/model.ckpt.


INFO:tensorflow:Loss for final step: 74179.7.


INFO:tensorflow:Loss for final step: 74179.7.


In [120]:
# Evaluation de l'efficacité du modèle
train_input_fn = tf.estimator.inputs.numpy_input_fn({"x": np.array(x_train)},
                                             np.array(y_train),
                                             batch_size=batch_size,
                                             num_epochs=iterations,
                                             shuffle=False)

eval_input_fn = tf.estimator.inputs.numpy_input_fn({"x": np.array(x_eval)},
                                             np.array(y_eval),
                                             batch_size=batch_size,
                                             num_epochs=iterations,
                                             shuffle=False)

train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics : %r"% train_metrics)
print("eval metrics : %r"% eval_metrics)

INFO:tensorflow:Starting evaluation at 2018-01-19-15:23:23


INFO:tensorflow:Starting evaluation at 2018-01-19-15:23:23


INFO:tensorflow:Restoring parameters from /tmp/tmp4stt7pgs/model.ckpt-1000


INFO:tensorflow:Restoring parameters from /tmp/tmp4stt7pgs/model.ckpt-1000


INFO:tensorflow:Finished evaluation at 2018-01-19-15:23:27


...

INFO:tensorflow:Finished evaluation at 2018-01-19-15:23:30


INFO:tensorflow:Finished evaluation at 2018-01-19-15:23:30


INFO:tensorflow:Saving dict for global step 1000: average_loss = 39.0804, global_step = 1000, loss = 78160.8


INFO:tensorflow:Saving dict for global step 1000: average_loss = 39.0804, global_step = 1000, loss = 78160.8


train metrics : {'average_loss': 36.765797, 'global_step': 1000, 'loss': 73531.594}
eval metrics : {'average_loss': 39.080402, 'global_step': 1000, 'loss': 78160.805}
